In [1]:
import os
import scipy.io
import numpy as np
from scipy import signal

# Set the path to the root directory where the "Control" folder is located
data_path = "/kaggle/input/control1"
lowcut = 0.4 
highcut = 100 
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')

for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []

for control_data_path in control_data:
    n_epochs = 70 # Number of total epochs available
    start_epoch = 35  # Starting epoch
    end_epoch = 70  # Ending epoch
    
    control_arr = []

    for i in range(start_epoch, end_epoch+1):
        epoch_path = os.path.join(control_data_path, f"trial{i}.mat")
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]        
        # Downsampling
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print(Control30[0].shape)


(36, 60, 1000)


In [2]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy

delta_band = (0.5, 4)
alpha_band = (8, 12)
beta_band = (12, 35)

all_control_power = []
scaler = MinMaxScaler()
epsilon = 1e-10

# Normalizing the features
for i in range(len(Control30)):
    control_data = Control30[i]
    control_power = []
    for epoch in control_data:
        epoch_power = []
        for channel_data in epoch:
            freq_spectrum = np.fft.fft(channel_data)
            power_spectrum = np.abs(freq_spectrum) ** 2

            alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
            beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
            delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

            channel_min_value = np.min(channel_data)
            channel_max_value = np.max(channel_data)

            # alpha_power_normalized = (alpha_power - channel_min_value) / (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value )))
            # beta_power_normalized = (beta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value) ))
            # delta_power_normalized = (delta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value)))

            power_spectrum_adjusted = power_spectrum + epsilon
            entropy_values = entropy(power_spectrum_adjusted)
            values=np.array([alpha_power,beta_power,delta_power,entropy_values])
            rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
            concatenated_data = np.concatenate((channel_data, rescaled_values))

            #concatenated_data = np.concatenate((channel_data, [alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values]))
            #print(alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values)
            epoch_power.append(concatenated_data.tolist())

        control_power.append(np.array(epoch_power))

    all_control_power.append(np.array(control_power))

all_control_power = np.array(all_control_power)
print(all_control_power.shape)
print(len(Control30))


(30, 36, 60, 1004)
30


In [3]:
import os
import scipy.io
import numpy as np
from scipy import signal
data_path = "/kaggle/input/concussed1"
lowcut = 0.4 
highcut = 100 
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed52 = []
for concussed_data_path in concussed_data:
    n_epochs = 70 # Number of total epochs available
    start_epoch = 35  # Starting epoch
    end_epoch = 70  # Ending epoch
    
    concussed_arr = []

    for i in range(start_epoch, end_epoch+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed52.append(np.array(concussed_arr))

Concussed52 = np.array(Concussed52)
print( Concussed52[0].shape)


(36, 60, 1000)


In [4]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy

delta_band = (0.5, 4)
alpha_band = (8, 12)
beta_band = (12, 35)

all_concussed_power = []
scaler = MinMaxScaler()
epsilon = 1e-10

# Normalizing the features within the range of the minimum and maximum values of the corresponding channel
for i in range(len(Concussed52)):
    concussed_data = Concussed52[i]
    concussed_power = []
    for epoch in concussed_data:
        epoch_power = []
        for channel_data in epoch:
            freq_spectrum = np.fft.fft(channel_data)
            power_spectrum = np.abs(freq_spectrum) ** 2

            alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
            beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
            delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

            channel_min_value = np.min(channel_data)
            channel_max_value = np.max(channel_data)

            # alpha_power_normalized = (alpha_power - channel_min_value) / (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value )))
            # beta_power_normalized = (beta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value) ))
            # delta_power_normalized = (delta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value)))

            power_spectrum_adjusted = power_spectrum + epsilon
            entropy_values = entropy(power_spectrum_adjusted)
            values=np.array([alpha_power,beta_power,delta_power,entropy_values])
            rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
            concatenated_data = np.concatenate((channel_data, rescaled_values))

            #concatenated_data = np.concatenate((channel_data, [alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values]))
            #print(alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values)
            epoch_power.append(concatenated_data.tolist())

        concussed_power.append(np.array(epoch_power))

    all_concussed_power.append(np.array(concussed_power))

all_concussed_power = np.array(all_concussed_power)
print(all_concussed_power.shape)
print(len(Concussed52))


(49, 36, 60, 1004)
49


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Step 1: Data Preparation
# Combine control and concussed data
data_combined = np.concatenate((all_control_power, all_concussed_power), axis=0)

# Create labels for binary classification
control_labels = np.zeros(all_control_power.shape[0])
concussed_labels = np.ones(all_concussed_power.shape[0])
labels_combined = np.concatenate((control_labels, concussed_labels))

# Step 2: Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data_combined, labels_combined, test_size=0.2, random_state=42)

# Step 3: Normalize the data
scaler = MinMaxScaler()
x_train_normalized = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_normalized = scaler.transform(x_test.reshape(x_test.shape[0], -1))

# Reshape the normalized data back to its original shape
x_train_normalized = x_train_normalized.reshape(x_train.shape)
x_test_normalized = x_test_normalized.reshape(x_test.shape)

# Verify the shapes of the training and testing sets
print("x_train shape:", x_train_normalized.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test_normalized.shape)
print("y_test shape:", y_test.shape)


x_train shape: (63, 36, 60, 1004)
y_train shape: (63,)
x_test shape: (16, 36, 60, 1004)
y_test shape: (16,)


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report


# Reshape the data
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)

# Standardize the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_reshaped)
x_test_scaled = scaler.transform(x_test_reshaped)

# Convert the labels to categorical (one-hot encoding)
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Reshape the flattened data back to 3D shape
x_train_scaled = x_train_scaled.reshape(x_train.shape[0], x_train.shape[1], -1)
x_test_scaled = x_test_scaled.reshape(x_test.shape[0], x_test.shape[1], -1)

# Define the number of folds
n_splits = 4

# Create a KFold object
kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store evaluation results
loss_scores = []
accuracy_scores = []
all_predictions = []
all_true_labels = []


# Perform k-fold cross-validation
for train_index, val_index in kf.split(x_train_scaled):
    # Split the data into training and validation sets for this fold
    x_train_fold, x_val_fold = x_train_scaled[train_index], x_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train_categorical[train_index], y_train_categorical[val_index]

    # Build the model architecture
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(x_train.shape[1], x_train_scaled.shape[2])))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(units=256, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(GRU(units=64, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(GRU(units=32)))
    model.add(Dropout(0.5))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=2, activation='softmax'))

    # Set the learning rate
    learning_rate = 0.0006
    optimizer = Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(x_train_fold, y_train_fold, validation_data=(x_val_fold, y_val_fold),
                        epochs=200, batch_size=128, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(x_val_fold, y_val_fold)
    loss_scores.append(loss)
    accuracy_scores.append(accuracy)
    
    val_predictions = model.predict(x_val_fold)
    val_predicted_labels = np.argmax(val_predictions, axis=1)
    val_true_labels = np.argmax(y_val_fold, axis=1)

    # Append predictions and true labels to the lists
    all_predictions.extend(val_predicted_labels)
    all_true_labels.extend(val_true_labels)


# Calculate the average scores across all folds
avg_loss = np.mean(loss_scores)
avg_accuracy = np.mean(accuracy_scores)

print('Average validation loss:', avg_loss)
print('Average validation accuracy:', avg_accuracy)

# Calculate the confusion matrix
cm = confusion_matrix(all_true_labels, all_predictions)
print('Confusion Matrix:')
print(cm)

# Calculate the classification report
class_report = classification_report(all_true_labels, all_predictions)
print('Classification Report:')
print(class_report)



/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

1/1 [==============================] - 5s 5s/step
Average validation loss: 0.5599031150341034
Average validation accuracy: 0.7114583402872086
Confusion Matrix:
[[10 13]
 [ 5 35]]
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.43      0.53        23
           1       0.73      0.88      0.80        40

    accuracy                           0.71        63
   macro avg       0.70      0.65      0.66        63
weighted avg       0.71      0.71      0.70        63



In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


y_pred_probs = model.predict(x_test_scaled)

y_pred = np.argmax(y_pred_probs, axis=1)

y_test_single = np.argmax(y_test_categorical, axis=1)

cm = confusion_matrix(y_test_single, y_pred)

report = classification_report(y_test_single, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)


1/1 [==============================] - 3s 3s/step
Confusion Matrix:
[[3 4]
 [0 9]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.43      0.60         7
           1       0.69      1.00      0.82         9

    accuracy                           0.75        16
   macro avg       0.85      0.71      0.71        16
weighted avg       0.83      0.75      0.72        16



In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
import numpy as np

# Reshape the input data
X_train = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], X_train.shape[2], X_train.shape[3]))
X_val = np.reshape(X_val, (X_val.shape[0]*X_val.shape[1], X_val.shape[2], X_val.shape[3]))

# Build the model
input_shape = (X_train.shape[1], X_train.shape[2])

model = tf.keras.Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]*X.shape[3]), return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(layers.Dropout(0.5))
model.add(GRU(256, kernel_regularizer=tf.keras.regularizers.l2(0.0001), return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()


NameError: name 'X_train' is not defined

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X = dataset
y = labels
Epoch = 100
batchSize = 256

k_folds = 2
kf = KFold(n_splits=k_folds, shuffle=True)

train_accuracies = []
val_accuracies = []
cm_total = np.zeros((2, 2))
y_preds = []

model = tf.keras.Sequential()
model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True), input_shape=(X.shape[1], X.shape[2]*X.shape[3])))
model.add(layers.Dropout(0.5))
model.add(layers.GRU(256, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.GRU(64, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.Bidirectional(layers.GRU(64, return_sequences=True)))
model.add(layers.Dropout(0.5))
model.add(layers.GRU(32))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer="adam", loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])
model.summary()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y_categorical[train_index], y_categorical[val_index]

    model.reset_states()

    history = model.fit(
        X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2]*X_train.shape[3]),
        y_train,
        epochs=Epoch,
        batch_size=batchSize,
        validation_data=(X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2]*X_val.shape[3]), y_val),
        verbose=1
    )

    train_loss, train_accuracy = model.evaluate(X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2]*X_train.shape[3]), y_train)
    val_loss, val_accuracy = model.evaluate(X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2]*X_val.shape[3]), y_val)

    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    y_pred = model.predict(X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2]*X_val.shape[3]))
    y_pred = np.argmax(y_pred, axis=1)
    y_preds.append(y_pred)

    cm = confusion_matrix(np.argmax(y_val, axis=1), y_pred)
    cm_total += cm

avg_train_accuracy = np.mean(train_accuracies)
avg_val_accuracy = np.mean(val_accuracies)
cm_avg = cm_total / k_folds

y_preds = np.concatenate(y_preds)

print('Confusion Matrix')
print(cm_total)
print('\n')

# Define target names
target_names = ['Control', 'Concussed']

# Calculate classification report
print('Classification Report')
print(classification_report(y_encoded, y_preds, target_names=target_names))


In [ ]:
avg_train_accuracy = np.mean(train_accuracies)
avg_val_accuracy = np.mean(val_accuracies)

print('Average Training Accuracy:', avg_train_accuracy)
print('Average Validation Accuracy:', avg_val_accuracy)


In [ ]:
print('Confusion Matrix')
print(cm_total)
print('\n')


In [ ]:
gjfdsl

In [ ]:
import os
import scipy.io
import numpy as np
from scipy import signal

# Set the path to the root directory where the "Control" folder is located
data_path = "/kaggle/input/control1"
lowcut = 0.4 
highcut = 50 
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')

for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []

for control_data_path in control_data:
    n_epochs = 105#len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    control_arr = []
    
    for i in range(1, n_epochs+1):
        epoch_path = os.path.join(control_data_path, f"trial{i}.mat")
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
                # Average referencing
#         average_potential = np.mean(mat_data1, axis=0, keepdims=True)
#         referenced_data = mat_data1 - average_potential
        
        # Bandpass filtering
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        
        # Apply notch filter
        f0 = 60  # Frequency to be removed (e.g., power line interference)
        Q = 30   # Quality factor
        w0 = f0 / (fs_original / 2)
        b, a = signal.iirnotch(w0, Q)
        filtered_data = signal.filtfilt(b, a, filtered_data, axis=-1)

        
        # Downsampling
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print(Control30[0].shape)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Reshape the Control30 data to prepare for scaling
num_trials, num_epochs, num_channels, num_samples = Control30.shape

Control30_scaled = np.zeros_like(Control30)

scaler = MinMaxScaler()

for i in range(num_trials):
    for j in range(num_epochs):
        # Flatten the 2D EEG data for scaling
        data = Control30[i, j].reshape(num_channels, num_samples)

        # Apply min-max scaling
        scaled_data = scaler.fit_transform(data)

        # Reshape the scaled data back to 2D
        Control30_scaled[i, j] = scaled_data.reshape(num_channels, num_samples)

print(Control30_scaled[0].shape)


In [ ]:
# import numpy as np
# import json
# from sklearn.preprocessing import MinMaxScaler
# from scipy.stats import entropy

# delta_band = (0.5, 4)
# alpha_band = (8, 12)
# beta_band = (12, 35)

# all_control_power = []
# scaler = MinMaxScaler()
# epsilon = 1e-10

# # Normalizing the features
# for i in range(len(Control30)):
#     control_data = Control30[i]
#     control_power = []
#     for epoch in control_data:
#         epoch_power = []
#         for channel_data in epoch:
#             freq_spectrum = np.fft.fft(channel_data)
#             power_spectrum = np.abs(freq_spectrum) ** 2

#             alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
#             beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
#             delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

#             channel_min_value = np.min(channel_data)
#             channel_max_value = np.max(channel_data)

#             # alpha_power_normalized = (alpha_power - channel_min_value) / (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value )))
#             # beta_power_normalized = (beta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value) ))
#             # delta_power_normalized = (delta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value)))

#             power_spectrum_adjusted = power_spectrum + epsilon
#             entropy_values = entropy(power_spectrum_adjusted)
#             values=np.array([alpha_power,beta_power,delta_power,entropy_values])
#             rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
#             concatenated_data = np.concatenate((channel_data, rescaled_values))

#             #concatenated_data = np.concatenate((channel_data, [alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values]))
#             #print(alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values)
#             epoch_power.append(concatenated_data.tolist())

#         control_power.append(np.array(epoch_power))

#     all_control_power.append(np.array(control_power))

# all_control_power = np.array(all_control_power)
# print(all_control_power.shape)
# print(len(Control30))


In [ ]:
import os
import scipy.io
import numpy as np
from scipy import signal
data_path = "/kaggle/input/concussed1"
lowcut = 0.4 
highcut = 50 
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed52 = []
for concussed_data_path in concussed_data:
    n_epochs = 105 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    concussed_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
#         # Average referencing
#         average_potential = np.mean(mat_data1, axis=0, keepdims=True)
#         referenced_data = mat_data1 - average_potential
        
        # Bandpass filtering
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        
        # Apply notch filter
        f0 = 60  # Frequency to be removed (e.g., power line interference)
        Q = 30   # Quality factor
        w0 = f0 / (fs_original / 2)
        b, a = signal.iirnotch(w0, Q)
        filtered_data = signal.filtfilt(b, a, filtered_data, axis=-1)
        
        # Downsampling
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed52.append(np.array(concussed_arr))

Concussed52 = np.array(Concussed52)
print( Concussed52[0].shape)

Concussed52 = Concussed52[:30]
print(Concussed52.shape)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Reshape the Control30 data to prepare for scaling
num_trials, num_epochs, num_channels, num_samples = Concussed52.shape

Concussed52_scaled = np.zeros_like(Concussed52)

scaler = MinMaxScaler()

for i in range(num_trials):
    for j in range(num_epochs):
        # Flatten the 2D EEG data for scaling
        data = Concussed52[i, j].reshape(num_channels, num_samples)

        # Apply min-max scaling
        scaled_data = scaler.fit_transform(data)

        # Reshape the scaled data back to 2D
        Concussed52_scaled[i, j] = scaled_data.reshape(num_channels, num_samples)

print(Concussed52_scaled[0].shape)


In [ ]:
# import numpy as np
# import json
# from sklearn.preprocessing import MinMaxScaler

# from scipy.stats import entropy

# delta_band = (0.5, 4)
# alpha_band = (8, 12)
# beta_band = (12, 35)

# all_concussed_power = []
# scaler = MinMaxScaler()
# epsilon = 1e-10

# # Normalizing the features within the range of the minimum and maximum values of the corresponding channel
# for i in range(len(Concussed52)):
#     concussed_data = Concussed52[i]
#     concussed_power = []
#     for epoch in concussed_data:
#         epoch_power = []
#         for channel_data in epoch:
#             freq_spectrum = np.fft.fft(channel_data)
#             power_spectrum = np.abs(freq_spectrum) ** 2

#             alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
#             beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
#             delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

#             channel_min_value = np.min(channel_data)
#             channel_max_value = np.max(channel_data)

#             # alpha_power_normalized = (alpha_power - channel_min_value) / (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value )))
#             # beta_power_normalized = (beta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value) ))
#             # delta_power_normalized = (delta_power - channel_min_value) /  (4*((channel_max_value-channel_min_value) * (channel_max_value-channel_min_value)))

#             power_spectrum_adjusted = power_spectrum + epsilon
#             entropy_values = entropy(power_spectrum_adjusted)
#             values=np.array([alpha_power,beta_power,delta_power,entropy_values])
#             rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
#             concatenated_data = np.concatenate((channel_data, rescaled_values))

#             #concatenated_data = np.concatenate((channel_data, [alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values]))
#             #print(alpha_power_normalized, beta_power_normalized, delta_power_normalized, entropy_values)
#             epoch_power.append(concatenated_data.tolist())

#         concussed_power.append(np.array(epoch_power))

#     all_concussed_power.append(np.array(concussed_power))

# all_concussed_power = np.array(all_concussed_power)
# print(all_concussed_power.shape)
# print(len(Concussed52))


In [ ]:
dataset = np.concatenate((Control30_scaled, Concussed52_scaled), axis=0)
labels = np.concatenate((np.zeros(len(Control30_scaled)), np.ones(len(Concussed52_scaled))))


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, GRU, Dense, Activation
from tensorflow.keras.optimizers import Adam
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

# Reshape the data to match the expected input shape of the model
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], -1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], -1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], -1))

# Define the input shape based on the reshaped data
input_shape = (X_train.shape[1], X_train.shape[2])

# Define the number of classes
num_classes = 2  # Assuming binary classification

# Convert the target variables to categorical format
y_train_categorical = tf.keras.utils.to_categorical(y_train, num_classes)
y_val_categorical = tf.keras.utils.to_categorical(y_val, num_classes)
y_test_categorical = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:

# Build the model
model = Sequential()

model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
model.add(Dropout(0.2))

model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))

model.add(Bidirectional(GRU(units=64, return_sequences=True)))
model.add(Dropout(0.2))

model.add(GRU(units=32))
model.add(Dropout(0.2))

model.add(Dense(units=16))
model.add(Activation('relu'))

model.add(Dense(units=num_classes))
model.add(Activation('softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"])
model.summary()

# # Train the model
# batch_size = 128
# epochs = 20
# model.fit(X_train, y_train_categorical, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val_categorical))

# # Evaluate the model on the testing data
# score = model.evaluate(X_test, y_test_categorical, verbose=0)
# print("Testing loss:", score[0])
# print("Testing accuracy:", score[1])


In [ ]:
import random

# Amplitude scaling augmentation function
def amplitude_scaling(data, factor_range=(0.8, 1.2)):
    scaling_factor = random.uniform(factor_range[0], factor_range[1])
    return data * scaling_factor

# Random noise addition augmentation function
def add_random_noise(data, noise_range=(-0.001, 0.001)):
    noise = np.random.uniform(noise_range[0], noise_range[1], size=data.shape)
    return data + noise

# Apply augmentation to the training data
X_train_augmented = []
for sample in X_train:
    scaled_sample = amplitude_scaling(sample)
    noisy_sample = add_random_noise(scaled_sample)
    X_train_augmented.append(noisy_sample)
X_train_augmented = np.array(X_train_augmented)

# Fit the model with the augmented training data
batch_size = 128
epochs = 200
model.fit(X_train_augmented, y_train_categorical, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val_categorical))


In [ ]:
# Evaluate the model on the testing data
score = model.evaluate(X_test, y_test_categorical, verbose=0)
print("Testing loss:", score[0])
print("Testing accuracy:", score[1])


In [ ]:
import numpy as np

# Define the time shift range in number of samples
time_shift_range = 10

# Generate augmented data by applying time shift
def apply_time_shift(data, shift_range):
    augmented_data = []
    for sample in data:
        shift_amount = np.random.randint(-shift_range, shift_range)
        augmented_sample = np.roll(sample, shift_amount, axis=2)
        augmented_data.append(augmented_sample)
    return np.array(augmented_data)

# Reshape the training data to (num_samples, num_channels, num_values)
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2] * X_train.shape[3]))

# Apply time shift augmentation to the training data
augmented_X_train_reshaped = apply_time_shift(X_train_reshaped, time_shift_range)

# Reshape the augmented data back to the original shape
augmented_X_train = np.reshape(augmented_X_train_reshaped, (augmented_X_train_reshaped.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]))

# Concatenate the original and augmented data
X_train_augmented = np.concatenate((X_train, augmented_X_train), axis=0)
y_train_augmented = np.concatenate((y_train, y_train), axis=0)

# Train the model with augmented data
model.fit(X_train_augmented, y_train_augmented, epochs=10, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, GRU, Dense, Activation
from tensorflow.keras.optimizers import Adam
import numpy as np

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(dataset, labels, test_size=0.1, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

# Reshape the data to match the expected input shape of the model
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], -1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], -1))

# Define the input shape based on the reshaped data
input_shape = (X_train.shape[1], X_train.shape[2])

# Define the number of classes
num_classes = 2  # Assuming binary classification

# Build the model
model = Sequential()

model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
model.add(Dropout(0.2))

model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))

model.add(Bidirectional(GRU(units=64, return_sequences=True)))
model.add(Dropout(0.2))

model.add(GRU(units=32))
model.add(Dropout(0.2))

model.add(Dense(units=16))
model.add(Activation('relu'))

model.add(Dense(units=num_classes))
model.add(Activation('softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])
model.summary()

# Train the model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

# Evaluate the model on the validation data
score = model.evaluate(X_val, y_val, verbose=0)
print("Validation loss:", score[0])
print("Validation accuracy:", score[1])


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
import numpy as np

# Reshape the input data
X_train = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], X_train.shape[2], X_train.shape[3]))
X_val = np.reshape(X_val, (X_val.shape[0]*X_val.shape[1], X_val.shape[2], X_val.shape[3]))

# Build the model
input_shape = (X_train.shape[1], X_train.shape[2])

model = tf.keras.Sequential()
model.add(layers.LSTM(64, input_shape=input_shape, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(64))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()


In [ ]:
print(Control30_scaled.shape)
print(Concussed52_scaled.shape)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Reshape the input data
X_train = X_train.reshape(-1, 60, 1000)
X_test = X_test.reshape(-1, 60, 1000)


# Define the ConcNet model architecture
model = keras.Sequential()

# Add a bidirectional LSTM layer with 64 units
model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(60, 1000)))
model.add(layers.Dropout(0.5))

# Add a GRU layer with 32 units
model.add(layers.GRU(units=32))
model.add(layers.Dropout(0.5))

# Add a dense layer with 16 units and ReLU activation
model.add(layers.Dense(units=16, activation='relu'))
model.add(layers.Dropout(0.5))

# Add the output layer with 1 unit and sigmoid activation for binary classification
model.add(layers.Dense(units=1, activation='sigmoid'))

# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.0001)

# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping and batch size of 256
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])


In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test))


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)


In [ ]:

dataset = np.concatenate((all_control_power, all_concussed_power), axis=0)
labels = np.concatenate((np.zeros(len(all_control_power)), np.ones(len(all_concussed_power))))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

X = np.concatenate((all_control_power, all_concussed_power), axis=0)

y = np.concatenate((np.zeros(all_control_power.shape[0] * 105), np.ones(all_concussed_power.shape[0] * 105)))

X = X.reshape(-1, X.shape[2], X.shape[3])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X.shape[1], X.shape[2])))
model.add(GRU(units=64))
model.add(Dropout(0.5))  # Add dropout layer with 20% dropout rate
model.add(Dense(units=1, activation='sigmoid'))

# Set learning rate for optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Print training and validation accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

_, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

target_names = ['Control', 'Concussed']
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))


In [ ]:
# num_control_samples = len(all_control_power)
# num_concussed_samples = len(all_concussed_power)

# # Randomly select a subset of samples from the larger class
# undersampled_concussed_indices = np.random.choice(
#     num_concussed_samples, size=num_control_samples, replace=False
# )
# undersampled_concussed_power = all_concussed_power[undersampled_concussed_indices]
# undersampled_concussed_labels = labels[num_control_samples:][undersampled_concussed_indices]

# # Combine the undersampled concussed data with the original control data
# undersampled_data = np.concatenate((all_control_power, undersampled_concussed_power), axis=0)
# undersampled_labels = np.concatenate((labels[:num_control_samples], undersampled_concussed_labels), axis=0)


In [ ]:
# print(undersampled_concussed_power.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(dataset, labels, test_size=0.1, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)




In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback_list = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
import numpy as np

# Reshape the input data
X_train = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], X_train.shape[2], X_train.shape[3]))
X_val = np.reshape(X_val, (X_val.shape[0]*X_val.shape[1], X_val.shape[2], X_val.shape[3]))

# Build the model
input_shape = (X_train.shape[1], X_train.shape[2])

model = tf.keras.Sequential()
model.add(layers.LSTM(64, input_shape=input_shape, return_sequences=True))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(64))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()


In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [ ]:
# from sklearn.model_selection import KFold
# from sklearn.metrics import classification_report, confusion_matrix
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers

# X = dataset
# y = labels
# Epoch = 200
# batchSize = 16

# k_folds = 2
# kf = KFold(n_splits=k_folds, shuffle=True)

# train_accuracies = []
# val_accuracies = []
# cm_total = np.zeros((2, 2))
# y_preds = []

# # Define EarlyStopping callback
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# for train_index, val_index in kf.split(X):
#     X_train, X_val = X[train_index], X[val_index]
#     y_train, y_val = y[train_index], y[val_index]

#     # Build and compile the model
#     model = tf.keras.Sequential()
#     model.add(layers.LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]*X_train.shape[3]), return_sequences=True))
#     model.add(layers.Dropout(0.7))
#     model.add(layers.LSTM(64))
#     model.add(layers.Dropout(0.7))
#     model.add(layers.Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

#     history = model.fit(
#         X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2]*X_train.shape[3]),
#         y_train,
#         epochs=Epoch,
#         batch_size=batchSize,
#         validation_data=(X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2]*X_val.shape[3]), y_val),
#         callbacks=[early_stop],  # Add the EarlyStopping callback
#         verbose=1
#     )

#     train_loss, train_accuracy = model.evaluate(X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2]*X_train.shape[3]), y_train)
#     val_loss, val_accuracy = model.evaluate(X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2]*X_val.shape[3]), y_val)

#     train_accuracies.append(train_accuracy)
#     val_accuracies.append(val_accuracy)

#     y_pred = model.predict(X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2]*X_val.shape[3]))
#     y_pred = np.round(y_pred).flatten()
#     y_preds.append(y_pred)

#     cm = confusion_matrix(y_val, y_pred)
#     cm_total += cm

# avg_train_accuracy = np.mean(train_accuracies)
# avg_val_accuracy = np.mean(val_accuracies)
# cm_avg = cm_total / k_folds

# y_preds = np.concatenate(y_preds)

# print('Confusion Matrix')
# print(cm_total)
# print('\n')

# # Define target names
# target_names = ['Control', 'Concussed']

# # Calculate classification report
# print('Classification Report')
# print(classification_report(y, y_preds, target_names=target_names))


In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, LSTM, Flatten, Dense

# # Assuming X and y are your EEG dataset and corresponding labels
# X = np.random.randn(105, 60, 1004)  # Replace with your actual data
# y = np.random.randint(0, 2, size=(105,))  # Replace with your actual labels

# # Reshape the data for 2D CNN
# X_2d_cnn = X.reshape(105, 60, 1004, 1)

# # Define 2D CNN model
# model_2d_cnn = Sequential()
# model_2d_cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(60, 1004, 1)))
# model_2d_cnn.add(Flatten())
# model_2d_cnn.add(Dense(128, activation='relu'))
# model_2d_cnn.add(Dense(1, activation='sigmoid'))
# model_2d_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model_2d_cnn.summary()

# # Fit the 2D CNN model
# model_2d_cnn.fit(X_2d_cnn, y, epochs=10, batch_size=32)

# # Reshape the data for LSTM
# X_lstm = X.reshape(105, 1004, 60)

# # Define LSTM model
# model_lstm = Sequential()
# model_lstm.add(LSTM(units=64, input_shape=(1004, 60)))
# model_lstm.add(Dense(128, activation='relu'))
# model_lstm.add(Dense(1, activation='sigmoid'))
# model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model_lstm.summary()

# # Fit the LSTM model
# model_lstm.fit(X_lstm, y, epochs=10, batch_size=32)


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

X = dataset
y = labels
Epoch = 100
batchSize = 16

k_folds = 2
kf = KFold(n_splits=k_folds, shuffle=True)

train_accuracies = []
val_accuracies = []
cm_total = np.zeros((2, 2))
y_preds = []

# Define EarlyStopping callback
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Build the CNN model
    cnn_model = models.Sequential()
    cnn_model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(105, 60, 1004)))
    cnn_model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
    cnn_model.add(layers.BatchNormalization())
    cnn_model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    cnn_model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
    cnn_model.add(layers.Dropout(0.2))
    cnn_model.add(layers.Flatten())

    # Build the LSTM model
    lstm_model = models.Sequential()
    lstm_model.add(layers.LSTM(units=128, input_shape=(13, 4)))
    lstm_model.add(layers.Dense(50, activation='relu'))

    # Combine the CNN and LSTM models
    combined = layers.concatenate([cnn_model.output, lstm_model.output])
    combined = layers.Dense(units=50, activation='relu')(combined)
    output_layer = layers.Dense(7, activation='softmax')(combined)

    # Create the final model
    model = models.Model(inputs=[cnn_model.input, lstm_model.input], outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        [X_train_cnn, X_train_lstm],
        y_train,
        epochs=Epoch,
        batch_size=batchSize,
        validation_data=([X_val_cnn, X_val_lstm], y_val),
        callbacks=[early_stop],
        verbose=1
    )

    train_loss, train_accuracy = model.evaluate(X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2]*X_train.shape[3]), y_train)
    val_loss, val_accuracy = model.evaluate(X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2]*X_val.shape[3]), y_val)


    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    y_pred = model.predict([X_val_cnn, X_val_lstm])
    y_pred = np.argmax(y_pred, axis=1)
    y_preds.append(y_pred)

    cm = confusion_matrix(np.argmax(y_val, axis=1), y_pred)
    cm_total += cm

avg_train_accuracy = np.mean(train_accuracies)
avg_val_accuracy = np.mean(val_accuracies)
cm_avg = cm_total / k_folds

y_preds = np.concatenate(y_preds)

print('Confusion Matrix')
print(cm_total)
print('\n')

# Define target names
target_names = ['Normal', 'Concussed']

# Calculate classification report
print('Classification Report')
print(classification_report(np.argmax(y, axis=1), y_preds, target_names=target_names))


In [ ]:
# Calculate the average accuracies
avg_train_accuracy = np.mean(train_accuracies)
avg_val_accuracy = np.mean(val_accuracies)

print('Average Training Accuracy:', avg_train_accuracy)
print('Average Validation Accuracy:', avg_val_accuracy)


In [ ]:
print('Confusion Matrix')
print(cm_total)
print('\n')

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

target_names = ['Normal', 'Concussed']
print('Confusion Matrix')
cm = confusion_matrix(true_labels, predicted_labels)
print(cm)
print('\n')

print('Classification Report')
print(classification_report(true_labels, predicted_labels, target_names=target_names))